# Traitements sur les vaccins

### Données

Trois jeux de données :

- juin COCONEL
- vague 1 & 2 de TRACTRUST (novembre et mai)

Plus le traitement médiatique dans les journaux avec Europresse pour le cadrage

In [1]:
import pandas as pd
import pyreadstat
import pyshs
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
import seaborn as sns


import researchpy as rp
from scipy.stats import chi2_contingency
from scipy import stats

def labels(x,meta):
    return meta.value_labels[meta.variable_to_label[x]]
def question(x,meta):
    return meta.column_names_to_labels[x]

### Chargement des trois jeux de données

Quelques limites de comparaison :
- pas la même question COCONEL et TRACTRUST
- pas de question sur la confiance dans COCONEL

In [2]:
data1, meta1 = pyreadstat.read_sav("Questionnaire chloro/Données/Data_CORO_V14.sav")
data2, meta2 = pyreadstat.read_sav('Questionnaire CS/conseilscientifique/SujetA.sav')
data3, meta3 = pyreadstat.read_sav('Questionnaire CS/conseilscientifique/Data_118230.sav')

data1["vague"] = "COCONEL"
data2["vague"] = "TRACTRUST1"
data3["vague"] = "TRACTRUST2"

# Ajout des variables recodées pour la vague 2

# Variables générales

data1["sexe"] = data1["SEXE"].replace({1:"Homme",2:"Femme"})
data2["sexe"] = data2["SEXE"].replace({1:"Homme",2:"Femme"})
data3["sexe"] = data3["SEXE"].replace({1:"Homme",2:"Femme"})

data1["age"] = pd.cut(data1["AGE"],[17,25,35,45,55,65,100])
data2["age"] = pd.cut(data2["AGE"],[17,25,35,45,55,65,100])
data3["age"] = pd.cut(data3["AGE"],[17,25,35,45,55,65,100])

data1["age2"] = pd.cut(data1["AGE"],[17,35,55,70,100],labels=["17-34","35-54","55-79","70-100"])
data2["age2"] = pd.cut(data2["AGE"],[17,35,55,70,100],labels=["17-34","35-54","55-79","70-100"])
data3["age2"] = pd.cut(data3["AGE"],[17,35,55,70,100],labels=["17-34","35-54","55-79","70-100"])

def reco(x):
    if x in [1,2]:
        return x
    if x in [3,4,5]:
        return "3-5"
    return "6 et +"
data1["foyer"] = data1["FOYER"].apply(reco)
data2["foyer"] = data2["FOYER"].apply(reco)
data3["foyer"] = data3["FOYER"].apply(reco)

reco = {1.0: 'Catholique',
 2.0: 'Catholique',
 3.0: 'Protestant ou évangélique',
 4.0: 'Musulman',
 5.0: 'Juif',
 6.0: 'Autre religion',
 7.0: 'Sans religion',
 8.0: 'Vous ne souhaitez pas répondre'}
#data1["religion"] = data1["REL1"].replace(reco)
data2["religion"] = data2["REL1"].replace(reco)
data3["religion"] = data3["REL1"].replace(reco)

data1["csp"] = data1["PI1"].replace(labels("PI1",meta1))
data2["csp"] = data2["PI1"].replace(labels("PI1",meta2))
data3["csp"] = data3["PI1"].replace(labels("PI1",meta3))

reco = {1:"1 - inf bac",2:"1 - inf bac",3:"1 - inf bac",4:"1 - inf bac",
        5:"1 - inf bac", 6:"1 - inf bac",7:"1 - inf bac",8:"2 - bac",9:"2 - bac",
        10:"3 - sup bac",11:"3 - sup bac"}
data1["diplome"] = data1["DIPL2"].replace(reco)
data2["diplome"] = data2["DIPL2"].replace(reco)
data3["diplome"] = data3["DIPL2"].replace(reco)

reco = {1.0: '1 - 0-1000€',2.0: '2 - 1000-2000€',
        3.0: '2 - 1000-2000€',4.0: '3 - 2000-4000€',
        5.0: '3 - 2000-4000€',6.0: '4 - 4000€ and more',7.0: '5 - NA'}
data1["revenus"] = data1["REV"].replace(reco)
data2["revenus"] = data2["REV"].replace(reco)
data3["revenus"] = data3["REV"].replace(reco)

reco = {1.0: '1 - Favorable',
 2.0: '1 - Favorable',
 3.0: '2 - Défavorable',
 4.0: '2 - Défavorable'}

data1["vac_general"] = data1['OA202'].replace(reco)
data2["vac_general"] = data2['QA32'].replace(reco)
data3["vac_general"] = data3['OA29'].replace(reco)


reco = {1.0: '1 - Tout à fait confiance',
 2.0: '2 - Plutôt confiance',
 3.0: '3 - Plutôt pas confiance',
 4.0: '5 - Pas du tout confiance',
 5.0: '5 - Vous ne savez pas'}


# Variables science
reco = {1.0: '1 - Yes', 2.0: '1 - Yes', 3.0: '2 - No', 4.0: '2 - No'}
data1["interetscience"] = data1["OA194"].replace(reco)
data2["interetscience"] = data2["QA18"].replace(reco)
data3["interetscience"] = data3["OA19"].replace(reco)

reco = {1.0: '1 - Plus de bien que de mal',
 2.0: '2 - A peu près autant de bien que de mal',
 3.0: '3 - Plus de mal que de bien',
 4.0: '4 - Vous ne savez pas'}
data1["apportscience"] = data1["OA195"].replace(reco)
reco = {1.0: '1 - Plus de bien que de mal',
 2.0: '3 - Plus de mal que de bien',
 3.0: '2 - A peu près autant de bien que de mal',
 4.0: '4 - Vous ne savez pas'}
data2["apportscience"] = data2["QA19"].replace(reco)
data3["apportscience"] = data3['OA20'].replace(reco)

reco = {1.0: '1 - Confiance',
 2.0: '1 - Confiance',
 3.0: '2 - Pas confiance',
 4.0: '2 - Pas confiance',
 5.0: '3 - Vous ne savez pas'}

#data1["confiancescience"] = data1['OA221'].replace(reco) ON A PAS CET ITEM DANS COCONEL
data2["confiancescience"] = data2['QA9_r5'].replace(reco)
data3["confiancescience"] = data3['OA8_r5'].replace(reco)

data2["confiancegouvernement"] = data2['QA9_r9'].replace(reco)
data3["confiancegouvernement"] = data3['OA8_r9'].replace(reco)



reco1 = {1:1,2:1,3:0,4:0,5:0}
data2["ind_confiance"] = (data2[["QA9_r%d"%i for i in range(1,13)]].replace(reco1).sum(axis=1)/(12*1))
data2["ind_confianceC"] = pd.qcut(data2["ind_confiance"],[0,0.25,0.5,0.75,1],["Q1 - Low","Q2","Q3","Q4 - High"])
data3["ind_confiance"] = (data3[["OA8_r%d"%i for i in range(1,13)]].replace(reco1).sum(axis=1)/(12*1))
data3["ind_confianceC"] = pd.qcut(data3["ind_confiance"],[0,0.25,0.5,0.75,1],["Q1 - Low","Q2","Q3","Q4 - High"])


# Variables politique

data1["parti"] = data1["PROX"].replace(labels("PROX",meta1))
data2["parti"] = data2["PROX"].replace(labels("PROX",meta2))
data3["parti"] = data3["PROX"].replace(labels("PROX",meta3))

reco = {1.0: '5 - Extrême gauche', 2.0: '5 - Extrême gauche', 3.0: '5 - Extrême gauche', 4.0: '5 - Extrême gauche',
 5.0: '3 - Gauche', 6.0: '3 - Gauche', 7.0: '3 - Gauche',
 8.0: '1 - Centre', 9.0: '1 - Centre', 10.0: '1 - Centre', 11.0: '1 - Centre',
 12.0: '4 - Droite', 13.0: '4 - Droite',
 14.0: '6 - Extrême droite', 15.0: '6 - Extrême droite', 16.0: '6 - Extrême droite', 17.0: '6 - Extrême droite',
 18.0: '7 - Une autre formation politique',
 19.0: '2 - Aucune formation politique'}

data1["PROXPARTI"] = data1["PROX"].replace(reco)
data2["PROXPARTI"] = data2["PROX"].replace(reco)
data3["PROXPARTI"] = data3["PROX"].replace(reco)


reco = {1:"Frequently",2:"Frequently",3:"Rarely",4:"Rarely",5:"Rarely"}
data1["infopol"] = data1["OA191"].replace(reco)
data2["infopol"] = data2["QA2"].replace(reco)
data3["infopol"] = data3["OA2"].replace(reco)

reco = {1:1,2:1,3:0,4:0,5:0}
data1["politisation"] = data1["OA190"].replace(reco) + data1["OA191"].replace(reco) + data1["OA193"].replace(reco) 
data1["politisationC"]  = data1["politisation"].replace({3:"3 - High",2:"2 - Average",1:"1 - Low",0:"1 - Low"})
data2["politisation"] = data2["QA1"].replace(reco) + data2["QA2"].replace(reco) + data2["QA4"].replace(reco) 
data2["politisationC"]  = data1["politisation"].replace({3:"3 - High",2:"2 - Average",1:"1 - Low",0:"1 - Low"})
data3["politisation"] = data3["OA1"].replace(reco) + data3["OA2"].replace(reco) + data3["OA4"].replace(reco) 
data3["politisationC"]  = data3["politisation"].replace({3:"3 - High",2:"2 - Average",1:"1 - Low",0:"1 - Low"})


# Variable chloroquine -> attention, dans la première vague on a fait un filtre sur le fait de savoir si oui ou non
# ils avaient entendu parler du débat

reco = {1:"1 - HC est un bon traitement",2:"2 - HC n'est pas un bon traitement",
        3:"3 - Conaissances insuffisantes",4:"4 - NSP"}
data1["HC_savoir"] = data1["OA200"].replace(labels("OA200",meta1))
# Croyance dans la chloroquine vague 1
reco = {3:"Incertains",4:"Sans avis",1:"Croyant",2:"Non croyant"}
data1["profil_hc"] = data1["OA201"].replace(reco)
data1["profil_hc"] = data1["profil_hc"].fillna("Ne connaissent pas")
data1["Croyant"] = data1["profil_hc"].apply(lambda x : 1 if x=="Croyant" else 0)

data1["HC"] = data1["OA201"].replace(reco)
data2["HC"] = data2["QA29"].replace(reco)
data3["HC"] = data3["OA27"].replace(reco)

reco = {1.0: 'Oui',2.0: 'Oui', 3.0: 'Non',4.0: 'Non'}
data1["vaccincovid"] = data1["OA27"].replace(reco)
data2["vaccincovid"] = data2["QA30"].replace(reco)
reco = {1.0: 'Oui',2.0: 'Oui', 3.0: 'Non',4.0: 'Non',5.0:"Oui"}
data3["vaccincovid"] = data3["OA28"].replace(reco)

reco = {1.0: 'Oui',2.0: 'Oui', 3.0: 'Non',4.0: 'Non'}
data1["vaccin"] = data1["OA202"].replace(reco)
data2["vaccin"] = data2["QA32"].replace(reco)
data3["vaccin"] = data3["OA29"].replace(reco)

# version complète

reco = {1.0: '1 - Oui, certainement',
 2.0: '1 - Oui, probablement',
 3.0: '2 - Non, probablement pas',
 4.0: '3 - Non, certainement pas'}
data1["vaccincovidF"] = data1["OA27"].replace(reco)
data2["vaccincovidF"] = data2["QA30"].replace(reco)
reco = {1.0: '1 - Oui certainement',
 2.0: '2 - Oui, probablement',
 3.0: '3 - Non, probablement pas',
 4.0: '4 - Non, certainement pas',
 5.0: '5 - Vous êtes déjà vacciné (une dose ou plus)'}
data3["vaccincovidF"] = data3["OA28"].replace(reco)


reco = {1.0: '1 - Très favorable',
 2.0: '2 - Plutôt favorable',
 3.0: '3 - Plutôt pas favorable',
 4.0: '4 - Pas du tout favorable'}
data1["vaccinF"] = data1["OA202"].replace(reco)
data2["vaccinF"] = data2["QA32"].replace(reco)
data3["vaccinF"] = data3["OA29"].replace(reco)

data1['confMinistereSecu'] = data1['OA219_r1'].replace({1:"1 - D'accord",2:"1 - D'accord",3:"2 - Pas d'accord",4:"2 - Pas d'accord",5:"3 - NSP"})
data1['confMinistereInfo'] = data1['OA218_r3'].replace({1:"1 - Confiance",2:"1 - Confiance",3:"2 - Pas confiance",4:"2 - Pas confiance"})

In [26]:
labels('OA218_r3',meta1)

{1.0: 'Très confiance',
 2.0: 'Plutôt confiance',
 3.0: 'Plutôt pas confiance',
 4.0: 'Pas du tout confiance'}

In [21]:
meta1.column_names_to_labels

{'ID': 'User Name',
 'COMPTEUR': 'COMPTEUR',
 'SEXE': 'SEXE - Vous êtes... ?',
 'AGE': 'AGE - Quel âge avez-vous ?  Merci de noter votre âge dans le cadre ci-dessous :',
 'RAGE1': 'RAGE1',
 'RAGE2': 'RAGE2',
 'RAGE3': 'RAGE3',
 'XRAGE': 'XRAGE',
 'jz2cinsee': 'jz2cinsee -',
 'AGGLOIFOP0': 'AGGLOIFOP0',
 'AGGLO': 'AGGLO',
 'TYPCOM': 'TYPCOM',
 'TAILCOM': 'TAILCOM',
 'DPT': 'DPT',
 'REG': 'REG',
 'REG2': 'REG2',
 'REG3': 'REG3',
 'REG22': 'REG22',
 'REG13': 'REG13',
 'REG12': 'REG12',
 'SITI': 'SITI - Quelle est votre situation ?',
 'PPIA': 'PPIA - Plus précisément, avant le confinement, quelle était votre profession principale ou la dernière profession principale que vous avez exercée ? Attention, si vous n’avez fait dans votre vie que des petits boulots (ex : job d’été, stage non',
 'PI0': 'PI0',
 'PIQ': 'PIQ',
 'PI1': 'PI1',
 'STC': 'STC - Avant le confinement, vous exerciez cette profession comme… ? Si vous exerciez plusieurs emplois, décrivez uniquement votre emploi principal.',
 'P

## non intention de se faire vacciner (plutôt pas ou certainement pas à la question sur les intentions de vaccination contre la covid vs le reste). Les variables indépendantes seraient : sexe, age, niveau de revenu, niveau d’éducation

In [28]:
# Variable binaire non
data1["vaccincovidN"] = data1["vaccincovid"].apply(lambda x : 1 if x == "Non" else 0)
data2["vaccincovidN"] = data2["vaccincovid"].apply(lambda x : 1 if x == "Non" else 0)
data3["vaccincovidN"] = data3["vaccincovid"].apply(lambda x : 1 if x == "Non" else 0)

tab = {
        "COCONEL":pyshs.regression_logistique(data1,"vaccincovidN",["sexe","age2","revenus","diplome"],"RED"),
        "TRACTRUST1":pyshs.regression_logistique(data2,"vaccincovidN",["sexe","age2","revenus","diplome"],"RED"),
        "TRACTRUST2":pyshs.regression_logistique(data2,"vaccincovidN",["sexe","age2","revenus","diplome"],"RED")}
pd.concat(tab,axis=1)

COCONEL                             \
                              Odds Ratio        p            IC 95%   
Variable   Modalité                                                   
.Intercept                          2.23   0.0***  2.23 [1.58-3.15]   
age2       17-34                     ref                              
           35-54                    0.69  0.002**  0.69 [0.55-0.87]   
           55-79                    0.38   0.0***  0.38 [0.29-0.51]   
           70-100                   0.21   0.0***  0.21 [0.15-0.29]   
diplome    1 - inf bac               ref                              
           2 - bac                  0.98    0.863  0.98 [0.75-1.27]   
           3 - sup bac              0.74   0.012*  0.74 [0.58-0.94]   
revenus    1 - 0-1000€               ref                              
           2 - 1000-2000€            0.7    0.04*  0.70 [0.50-0.98]   
           3 - 2000-4000€           0.55   0.0***  0.55 [0.39-0.76]   
           4 - 4000€ and more       0.38   0.0***  0.38 [0.24-0.60]   
           5 - NA                   0.59   0.013*  0.59 [0.39-0.90]   
sexe       Femme                     ref                              
           Homme                    0.66   0.0***  0.66 [0.55-0.80]   

                              TRACTRUST1                             \
                              Odds Ratio        p            IC 95%   
Variable   Modalité                                                   
.Intercept                          3.88   0.0***  3.88 [2.26-6.64]   
age2       17-34                     ref                              
           35-54                    0.62  0.009**  0.62 [0.44-0.89]   
           55-79                    0.27   0.0***  0.27 [0.18-0.40]   
           70-100                   0.16   0.0***  0.16 [0.10-0.26]   
diplome    1 - inf bac               ref                              
           2 - bac                  0.87    0.464  0.87 [0.60-1.27]   
           3 - sup bac              0.58  0.002**  0.58 [0.41-0.82]   
revenus    1 - 0-1000€               ref                              
           2 - 1000-2000€           1.24     0.41  1.24 [0.74-2.09]   
           3 - 2000-4000€           1.22    0.456  1.22 [0.73-2.04]   
           4 - 4000€ and more       1.01    0.982  1.01 [0.51-1.97]   
           5 - NA                   0.92    0.799  0.92 [0.51-1.69]   
sexe       Femme                     ref                              
           Homme                    0.44   0.0***  0.44 [0.34-0.58]   

                              TRACTRUST2                             
                              Odds Ratio        p            IC 95%  
Variable   Modalité                                                  
.Intercept                          3.88   0.0***  3.88 [2.26-6.64]  
age2       17-34                     ref                             
           35-54                    0.62  0.009**  0.62 [0.44-0.89]  
           55-79                    0.27   0.0***  0.27 [0.18-0.40]  
           70-100                   0.16   0.0***  0.16 [0.10-0.26]  
diplome    1 - inf bac               ref                             
           2 - bac                  0.87    0.464  0.87 [0.60-1.27]  
           3 - sup bac              0.58  0.002**  0.58 [0.41-0.82]  
revenus    1 - 0-1000€               ref                             
           2 - 1000-2000€           1.24     0.41  1.24 [0.74-2.09]  
           3 - 2000-4000€           1.22    0.456  1.22 [0.73-2.04]  
           4 - 4000€ and more       1.01    0.982  1.01 [0.51-1.97]  
           5 - NA                   0.92    0.799  0.92 [0.51-1.69]  
sexe       Femme                     ref                             
           Homme                    0.44   0.0***  0.44 [0.34-0.58]

## 2. Spire et Bajos mettent pas ensemble ceux qui n'ont plutot pas l'intention de se faire vacciner et ceux qui disent "certainement pas". Pour voir si leurs résultats sont spécifiques à la période ou s'ils traduisent une constante pour la période, j'aimerais faire la même régression que ci-dessus sur les 3 BDD sauf que ce serait les "certainement pas" vs le reste.

In [24]:
reco = {1.0: 0,2.0: 0, 3.0: 0,4.0: 1}
data1["vaccincovidCP"] = data1["OA27"].replace(reco)
data2["vaccincovidCP"] = data2["QA30"].replace(reco)
reco = {1.0: 0,2.0: 0, 3.0: 0,4.0: 1,5.0:0}
data3["vaccincovidCP"] = data3["OA28"].replace(reco)

In [29]:
tab = {
        "COCONEL":pyshs.tri_a_plat(data1,"vaccincovidCP","RED"),
        "TRACTRUST1":pyshs.tri_a_plat(data2,"vaccincovidCP","RED"),
        "TRACTRUST2":pyshs.tri_a_plat(data3,"vaccincovidCP","RED")}
pd.concat(tab,axis=1)

COCONEL                        TRACTRUST1                  \
      Effectif redressé Pourcentage (%) Effectif redressé Pourcentage (%)   
0.0              1734.9            86.5             768.7            75.8   
1.0               271.1            13.5             245.3            24.2   
Total            2006.0           100.0            1014.0           100.0   

             TRACTRUST2                  
      Effectif redressé Pourcentage (%)  
0.0               885.4            88.1  
1.0               119.6            11.9  
Total            1005.0           100.0

Il y a un truc étrange sur la vague 1 de TRACTRUST (quasiment le double)

In [27]:
tab = {
        "COCONEL":pyshs.regression_logistique(data1,"vaccincovidCP",["sexe","age2","revenus","diplome"],"RED"),
        "TRACTRUST1":pyshs.regression_logistique(data2,"vaccincovidCP",["sexe","age2","revenus","diplome"],"RED"),
        "TRACTRUST2":pyshs.regression_logistique(data2,"vaccincovidCP",["sexe","age2","revenus","diplome"],"RED")}
pd.concat(tab,axis=1)

COCONEL                             \
                              Odds Ratio        p            IC 95%   
Variable   Modalité                                                   
.Intercept                          0.62   0.021*  0.62 [0.41-0.93]   
age2       17-34                     ref                              
           35-54                    1.14    0.402  1.14 [0.84-1.55]   
           55-79                    0.54  0.003**  0.54 [0.36-0.81]   
           70-100                   0.18   0.0***  0.18 [0.09-0.33]   
diplome    1 - inf bac               ref                              
           2 - bac                   0.7   0.049*  0.70 [0.49-1.00]   
           3 - sup bac              0.52   0.0***  0.52 [0.37-0.73]   
revenus    1 - 0-1000€               ref                              
           2 - 1000-2000€           0.51  0.001**  0.51 [0.34-0.77]   
           3 - 2000-4000€           0.44   0.0***  0.44 [0.30-0.66]   
           4 - 4000€ and more       0.22   0.0***  0.22 [0.11-0.44]   
           5 - NA                   0.54   0.019*  0.54 [0.33-0.90]   
sexe       Femme                     ref                              
           Homme                    0.63  0.001**  0.63 [0.48-0.83]   

                              TRACTRUST1                             \
                              Odds Ratio        p            IC 95%   
Variable   Modalité                                                   
.Intercept                          1.34    0.285  1.34 [0.78-2.31]   
age2       17-34                     ref                              
           35-54                    0.72    0.072  0.72 [0.50-1.03]   
           55-79                    0.33   0.0***  0.33 [0.21-0.53]   
           70-100                   0.17   0.0***  0.17 [0.09-0.31]   
diplome    1 - inf bac               ref                              
           2 - bac                  0.83    0.375  0.83 [0.55-1.25]   
           3 - sup bac              0.49  0.001**  0.49 [0.33-0.74]   
revenus    1 - 0-1000€               ref                              
           2 - 1000-2000€           0.82    0.456  0.82 [0.48-1.38]   
           3 - 2000-4000€           0.68     0.16  0.68 [0.40-1.16]   
           4 - 4000€ and more       0.49    0.085  0.49 [0.21-1.10]   
           5 - NA                   0.65    0.192  0.65 [0.35-1.24]   
sexe       Femme                     ref                              
           Homme                    0.44   0.0***  0.44 [0.32-0.61]   

                              TRACTRUST2                             
                              Odds Ratio        p            IC 95%  
Variable   Modalité                                                  
.Intercept                          1.34    0.285  1.34 [0.78-2.31]  
age2       17-34                     ref                             
           35-54                    0.72    0.072  0.72 [0.50-1.03]  
           55-79                    0.33   0.0***  0.33 [0.21-0.53]  
           70-100                   0.17   0.0***  0.17 [0.09-0.31]  
diplome    1 - inf bac               ref                             
           2 - bac                  0.83    0.375  0.83 [0.55-1.25]  
           3 - sup bac              0.49  0.001**  0.49 [0.33-0.74]  
revenus    1 - 0-1000€               ref                             
           2 - 1000-2000€           0.82    0.456  0.82 [0.48-1.38]  
           3 - 2000-4000€           0.68     0.16  0.68 [0.40-1.16]  
           4 - 4000€ and more       0.49    0.085  0.49 [0.21-1.10]  
           5 - NA                   0.65    0.192  0.65 [0.35-1.24]  
sexe       Femme                     ref                             
           Homme                    0.44   0.0***  0.44 [0.32-0.61]

## 3.Spire et Bajos font la même chose pour les attitudes à l'égard de la vaccination en général. L'item est "êtes vous favorable aux vaccins en général' et ils font les "pas du tout" vs le reste. Je veux bien que l'on fasse pareil sur les 3 BDD (toujours avec le  sexe, age, niveau de revenu, niveau d’éducation comme variables indépendantes)

In [38]:
reco = {1.0: 0,2.0: 0, 3.0: 0,4.0: 1}
data1["vaccinDF"] = data1["OA202"].replace(reco)
data2["vaccinDF"] = data2["QA32"].replace(reco)
data3["vaccinDF"] = data3["OA29"].replace(reco)

In [39]:
tab = {
        "COCONEL":pyshs.tri_a_plat(data1,"vaccinDF","RED"),
        "TRACTRUST1":pyshs.tri_a_plat(data2,"vaccinDF","RED"),
        "TRACTRUST2":pyshs.tri_a_plat(data3,"vaccinDF","RED")}
pd.concat(tab,axis=1)

COCONEL                        TRACTRUST1                  \
      Effectif redressé Pourcentage (%) Effectif redressé Pourcentage (%)   
0.0              1835.9            91.5             932.8            92.0   
1.0               170.1             8.5              81.2             8.0   
Total            2006.0           100.0            1014.0           100.0   

             TRACTRUST2                  
      Effectif redressé Pourcentage (%)  
0.0               927.7            92.3  
1.0                77.3             7.7  
Total            1005.0           100.0

In [40]:
tab = {
        "COCONEL":pyshs.regression_logistique(data1,"vaccinDF",["sexe","age2","revenus","diplome"],"RED"),
        "TRACTRUST1":pyshs.regression_logistique(data2,"vaccinDF",["sexe","age2","revenus","diplome"],"RED"),
        "TRACTRUST2":pyshs.regression_logistique(data2,"vaccinDF",["sexe","age2","revenus","diplome"],"RED")}
pd.concat(tab,axis=1)

COCONEL                             \
                              Odds Ratio        p            IC 95%   
Variable   Modalité                                                   
.Intercept                          0.33   0.0***  0.33 [0.20-0.53]   
age2       17-34                     ref                              
           35-54                     1.0    0.983  1.00 [0.69-1.45]   
           55-79                    0.51  0.007**  0.51 [0.31-0.84]   
           70-100                   0.23   0.0***  0.23 [0.11-0.45]   
diplome    1 - inf bac               ref                              
           2 - bac                  0.61    0.03*  0.61 [0.39-0.95]   
           3 - sup bac              0.45   0.0***  0.45 [0.29-0.69]   
revenus    1 - 0-1000€               ref                              
           2 - 1000-2000€           0.54   0.014*  0.54 [0.33-0.88]   
           3 - 2000-4000€           0.55   0.013*  0.55 [0.34-0.88]   
           4 - 4000€ and more        0.2  0.001**  0.20 [0.07-0.52]   
           5 - NA                   0.66    0.169  0.66 [0.36-1.20]   
sexe       Femme                     ref                              
           Homme                    0.77    0.118  0.77 [0.56-1.07]   

                              TRACTRUST1                           TRACTRUST2  \
                              Odds Ratio       p            IC 95% Odds Ratio   
Variable   Modalité                                                             
.Intercept                          0.19  0.0***  0.19 [0.08-0.43]       0.19   
age2       17-34                     ref                                  ref   
           35-54                    1.21   0.504  1.21 [0.69-2.13]       1.21   
           55-79                    0.52   0.094  0.52 [0.24-1.12]       0.52   
           70-100                   0.33  0.026*  0.33 [0.12-0.88]       0.33   
diplome    1 - inf bac               ref                                  ref   
           2 - bac                  0.76   0.375  0.76 [0.41-1.39]       0.76   
           3 - sup bac              0.42  0.013*  0.42 [0.22-0.83]       0.42   
revenus    1 - 0-1000€               ref                                  ref   
           2 - 1000-2000€           0.98   0.951  0.98 [0.46-2.09]       0.98   
           3 - 2000-4000€            0.6   0.219  0.60 [0.27-1.35]        0.6   
           4 - 4000€ and more       0.19   0.091  0.19 [0.03-1.30]       0.19   
           5 - NA                   1.04   0.934  1.04 [0.42-2.56]       1.04   
sexe       Femme                     ref                                  ref   
           Homme                    0.78   0.325  0.78 [0.48-1.27]       0.78   

                                                         
                                    p            IC 95%  
Variable   Modalité                                      
.Intercept                     0.0***  0.19 [0.08-0.43]  
age2       17-34                                         
           35-54                0.504  1.21 [0.69-2.13]  
           55-79                0.094  0.52 [0.24-1.12]  
           70-100              0.026*  0.33 [0.12-0.88]  
diplome    1 - inf bac                                   
           2 - bac              0.375  0.76 [0.41-1.39]  
           3 - sup bac         0.013*  0.42 [0.22-0.83]  
revenus    1 - 0-1000€                                   
           2 - 1000-2000€       0.951  0.98 [0.46-2.09]  
           3 - 2000-4000€       0.219  0.60 [0.27-1.35]  
           4 - 4000€ and more   0.091  0.19 [0.03-1.30]  
           5 - NA               0.934  1.04 [0.42-2.56]  
sexe       Femme                                         
           Homme                0.325  0.78 [0.48-1.27]

## Ils ne concernent que les vagues de tractrust. Je voudrais pour chacune des 2 un tableau tout simple croisant les réponses à la question "êtes vous favorables aux vaccins en général' et celles à la question "êtes-vous défavorables à certains vaccins en particulier".

In [46]:
reco = {1.0: '1 - Très favorable',
 2.0: '2 - Plutôt favorable',
 3.0: '3 - Plutôt pas favorable',
 4.0: '4 - Pas du tout favorable'}
data2["vaccin"] = data2["QA32"].replace(reco)
data3["vaccin"] = data3["OA29"].replace(reco)
reco = {1.0: '1 - Oui', 2.0: 'Non', 3.0: 'Vous ne savez pas'}
data2["vaccinpar"] = data2["QA33"].replace(reco)
data3["vaccinpar"] = data3["OA30"].replace(reco)

In [47]:
pyshs.tableau_croise(data2,"vaccin","vaccinpar","RED")

vaccinpar,1 - Oui,Non,Vous ne savez pas,All
vaccin,,,,
1 - Très favorable,32.5 (13.5%),188.6 (78.1%),20.3 (8.4%),241.5 (100%)
2 - Plutôt favorable,139.5 (28.7%),253.5 (52.1%),93.1 (19.2%),486.1 (100%)
3 - Plutôt pas favorable,139.0 (67.7%),43.4 (21.2%),22.8 (11.1%),205.2 (100%)
4 - Pas du tout favorable,40.0 (49.3%),24.6 (30.3%),16.6 (20.4%),81.2 (100%)
All,351.1 (34.6%),510.1 (50.3%),152.8 (15.1%),1014.0 (100%)


In [48]:
pyshs.tableau_croise(data3,"vaccin","vaccinpar","RED")

vaccinpar,1 - Oui,Non,Vous ne savez pas,All
vaccin,,,,
1 - Très favorable,63.0 (17.5%),285.2 (79.2%),12.0 (3.3%),360.2 (100%)
2 - Plutôt favorable,128.2 (33.1%),207.0 (53.4%),52.4 (13.5%),387.5 (100%)
3 - Plutôt pas favorable,104.0 (57.8%),47.9 (26.6%),28.0 (15.6%),179.9 (100%)
4 - Pas du tout favorable,48.2 (62.4%),18.2 (23.5%),10.9 (14.1%),77.3 (100%)
All,343.4 (34.2%),558.3 (55.6%),103.3 (10.3%),1005.0 (100%)


### Tableaux croisés du 23072021

COCONEL

In [3]:
pyshs.tableau_croise(data1,"vaccinF","vaccincovidF","RED")

vaccincovidF,"1 - Oui, certainement","1 - Oui, probablement","2 - Non, probablement pas","3 - Non, certainement pas",All
vaccinF,,,,,
1 - Très favorable,328.0 (60.0%),167.2 (30.6%),44.4 (8.1%),6.8 (1.2%),546.3 (100%)
2 - Plutôt favorable,120.3 (12.9%),550.9 (58.9%),217.4 (23.3%),46.3 (5.0%),934.9 (100%)
3 - Plutôt pas favorable,5.1 (1.4%),82.5 (23.3%),183.5 (51.7%),83.6 (23.6%),354.7 (100%)
4 - Pas du tout favorable,2.5 (1.5%),7.5 (4.4%),25.7 (15.1%),134.4 (79.0%),170.1 (100%)
All,455.9 (22.7%),808.0 (40.3%),471.0 (23.5%),271.1 (13.5%),2006.0 (100%)


TRACTRUST 1

In [4]:
pyshs.tableau_croise(data2,"vaccinF","vaccincovidF","RED")

vaccincovidF,"1 - Oui, certainement","1 - Oui, probablement","2 - Non, probablement pas","3 - Non, certainement pas",All
vaccinF,,,,,
1 - Très favorable,117.1 (48.5%),82.5 (34.2%),23.6 (9.8%),18.3 (7.6%),241.5 (100%)
2 - Plutôt favorable,40.4 (8.3%),191.3 (39.4%),180.4 (37.1%),74.0 (15.2%),486.1 (100%)
3 - Plutôt pas favorable,2.1 (1.0%),28.7 (14.0%),85.8 (41.8%),88.7 (43.2%),205.2 (100%)
4 - Pas du tout favorable,0.6 (0.7%),5.1 (6.3%),11.2 (13.8%),64.3 (79.2%),81.2 (100%)
All,160.1 (15.8%),307.5 (30.3%),301.0 (29.7%),245.3 (24.2%),1014.0 (100%)


TRACTRUST 2

In [5]:
pyshs.tableau_croise(data3,"vaccinF","vaccincovidF","RED")

vaccincovidF,1 - Oui certainement,"2 - Oui, probablement","3 - Non, probablement pas","4 - Non, certainement pas",5 - Vous êtes déjà vacciné (une dose ou plus),All
vaccinF,,,,,,
1 - Très favorable,99.7 (27.7%),9.3 (2.6%),11.6 (3.2%),13.2 (3.7%),226.4 (62.9%),360.2 (100%)
2 - Plutôt favorable,59.7 (15.4%),90.3 (23.3%),33.9 (8.7%),33.8 (8.7%),169.8 (43.8%),387.5 (100%)
3 - Plutôt pas favorable,18.7 (10.4%),43.1 (23.9%),47.3 (26.3%),34.7 (19.3%),36.2 (20.1%),179.9 (100%)
4 - Pas du tout favorable,5.5 (7.1%),4.8 (6.2%),15.7 (20.3%),38.0 (49.2%),13.3 (17.2%),77.3 (100%)
All,183.7 (18.3%),147.5 (14.7%),108.5 (10.8%),119.6 (11.9%),445.7 (44.3%),1005.0 (100%)


### Tableaux croisés du 02082021

- Un tableau où, au sein de chaque formation partisane, on voit les intentions de vaccination contre la covid en fonction du degré d'engagement politique (on avait fait un composite de trois variables).
- la même chose sur l'attitude à l'égard de la vaccination en général
- les deux mêmes tableaux mais en remplaçant le niveau d'engagement politique par le niveau d'éducation (on l'avait trichotomisé c'est bien ça?)
- les deux mêmes tableaux mais en remplaçant le niveau d'engagement politique par la confiance dans le gouvernement pour gérer la covid (en oui/non).

Croisement avec la politisation (degré d'engagement), d'abord vac COVID, puis vac général

In [22]:
pyshs.tableau_croise_controle(data1,"PROXPARTI", "politisationC","vaccincovid","RED").sort_index()

vaccincovid                                                            Non  \
PROXPARTI                                     politisationC                  
1 - Centre (p = 0.057)                        1 - Low         14.2 (26.0%)   
                                              2 - Average     16.0 (27.8%)   
                                              3 - High        26.2 (15.4%)   
                                              All             56.4 (19.9%)   
2 - Aucune formation politique (p = 0.067)    1 - Low        228.6 (46.6%)   
                                              2 - Average     41.9 (37.4%)   
                                              3 - High        52.8 (37.9%)   
                                              All            323.3 (43.6%)   
3 - Gauche (p = 0.044)                        1 - Low         35.6 (40.5%)   
                                              2 - Average      7.2 (22.6%)   
                                              3 - High        31.3 (26.0%)   
                                              All             74.1 (30.8%)   
4 - Droite (p = 0.001)                        1 - Low         14.8 (30.9%)   
                                              2 - Average     16.1 (32.3%)   
                                              3 - High        16.2 (12.2%)   
                                              All             47.1 (20.5%)   
5 - Extrême gauche (p = 0.050)                1 - Low         36.4 (46.1%)   
                                              2 - Average     20.3 (41.9%)   
                                              3 - High        19.2 (27.2%)   
                                              All             75.8 (38.3%)   
6 - Extrême droite (p = 0.019)                1 - Low         74.9 (60.4%)   
                                              2 - Average     40.0 (58.0%)   
                                              3 - High        39.3 (42.0%)   
                                              All            154.2 (53.8%)   
7 - Une autre formation politique (p = 0.064) 1 - Low          3.8 (52.8%)   
                                              2 - Average      5.9 (66.3%)   
                                              3 - High         1.6 (15.4%)   
                                              All             11.3 (42.6%)   

vaccincovid                                                            Oui  \
PROXPARTI                                     politisationC                  
1 - Centre (p = 0.057)                        1 - Low         40.5 (74.0%)   
                                              2 - Average     41.5 (72.2%)   
                                              3 - High       144.3 (84.6%)   
                                              All            226.4 (80.1%)   
2 - Aucune formation politique (p = 0.067)    1 - Low        261.9 (53.4%)   
                                              2 - Average     70.1 (62.6%)   
                                              3 - High        86.6 (62.1%)   
                                              All            418.7 (56.4%)   
3 - Gauche (p = 0.044)                        1 - Low         52.2 (59.5%)   
                                              2 - Average     24.7 (77.4%)   
                                              3 - High        89.2 (74.0%)   
                                              All            166.1 (69.2%)   
4 - Droite (p = 0.001)                        1 - Low         33.1 (69.1%)   
                                              2 - Average     33.7 (67.7%)   
                                              3 - High       116.2 (87.8%)   
                                              All            182.9 (79.5%)   
5 - Extrême gauche (p = 0.050)                1 - Low         42.5 (53.9%)   
                                              2 - Average     28.2 (58.1%)   
                                              3 - High        51.4 (72.8%)   
                     

In [18]:
pyshs.tableau_croise_controle(data1,"PROXPARTI", "politisationC","vaccin","RED").sort_index()

vaccin                                                                 Non  \
PROXPARTI                                     politisationC                  
1 - Centre (p = 0.104)                        1 - Low         12.1 (22.1%)   
                                              2 - Average      8.1 (14.1%)   
                                              3 - High        18.4 (10.8%)   
                                              All             38.6 (13.7%)   
2 - Aucune formation politique (p = 0.013)    1 - Low        175.3 (35.7%)   
                                              2 - Average     36.8 (32.8%)   
                                              3 - High        31.3 (22.5%)   
                                              All            243.4 (32.8%)   
3 - Gauche (p = 0.001)                        1 - Low         28.7 (32.7%)   
                                              2 - Average       2.1 (6.6%)   
                                              3 - High        18.8 (15.6%)   
                                              All             49.6 (20.6%)   
4 - Droite (p = 0.056)                        1 - Low           4.3 (9.0%)   
                                              2 - Average      8.8 (17.7%)   
                                              3 - High          8.1 (6.1%)   
                                              All              21.3 (9.3%)   
5 - Extrême gauche (p = 0.230)                1 - Low         27.0 (34.2%)   
                                              2 - Average     16.1 (33.2%)   
                                              3 - High        15.7 (22.2%)   
                                              All             58.8 (29.7%)   
6 - Extrême droite (p = 0.010)                1 - Low         57.1 (46.0%)   
                                              2 - Average     27.5 (39.9%)   
                                              3 - High        24.4 (26.1%)   
                                              All            109.1 (38.1%)   
7 - Une autre formation politique (p = 0.533) 1 - Low          1.9 (26.4%)   
                                              2 - Average      1.5 (16.9%)   
                                              3 - High          0.7 (6.8%)   
                                              All              4.1 (15.5%)   

vaccin                                                                 Oui  \
PROXPARTI                                     politisationC                  
1 - Centre (p = 0.104)                        1 - Low         42.6 (77.9%)   
                                              2 - Average     49.5 (85.9%)   
                                              3 - High       152.1 (89.2%)   
                                              All            244.1 (86.3%)   
2 - Aucune formation politique (p = 0.013)    1 - Low        315.3 (64.3%)   
                                              2 - Average     75.3 (67.2%)   
                                              3 - High       108.0 (77.5%)   
                                              All            498.6 (67.2%)   
3 - Gauche (p = 0.001)                        1 - Low         59.1 (67.3%)   
                                              2 - Average     29.8 (93.4%)   
                                              3 - High       101.7 (84.4%)   
                                              All            190.6 (79.4%)   
4 - Droite (p = 0.056)                        1 - Low         43.5 (91.0%)   
                                              2 - Average     41.0 (82.3%)   
                                              3 - High       124.3 (93.9%)   
                                              All            208.7 (90.7%)   
5 - Extrême gauche (p = 0.230)                1 - Low         51.9 (65.8%)   
                                              2 - Average     32.4 (66.8%)   
                                              3 - High        54.9 (77.8%)   
                     

Croisement avec les diplomes

In [23]:
pyshs.tableau_croise_controle(data1,"PROXPARTI", "diplome","vaccincovid","RED").sort_index()

vaccincovid                                                          Non  \
PROXPARTI                                     diplome                      
1 - Centre (p = 0.204)                        1 - inf bac   20.2 (17.6%)   
                                              2 - bac       12.2 (30.2%)   
                                              3 - sup bac   23.9 (18.8%)   
                                              All           56.4 (19.9%)   
2 - Aucune formation politique (p = 0.657)    1 - inf bac  179.4 (45.0%)   
                                              2 - bac       62.9 (43.3%)   
                                              3 - sup bac   81.0 (41.0%)   
                                              All          323.3 (43.6%)   
3 - Gauche (p = 0.800)                        1 - inf bac   28.9 (28.6%)   
                                              2 - bac       13.5 (33.7%)   
                                              3 - sup bac   31.7 (32.0%)   
                                              All           74.1 (30.8%)   
4 - Droite (p = 0.003)                        1 - inf bac   10.6 (10.8%)   
                                              2 - bac       16.1 (34.3%)   
                                              3 - sup bac   20.4 (23.9%)   
                                              All           47.1 (20.5%)   
5 - Extrême gauche (p = 0.755)                1 - inf bac   36.1 (35.9%)   
                                              2 - bac       19.5 (39.6%)   
                                              3 - sup bac   20.3 (42.0%)   
                                              All           75.8 (38.3%)   
6 - Extrême droite (p = 0.446)                1 - inf bac   99.4 (52.4%)   
                                              2 - bac       29.8 (62.1%)   
                                              3 - sup bac   25.0 (51.1%)   
                                              All          154.2 (53.8%)   
7 - Une autre formation politique (p = 0.283) 1 - inf bac    5.7 (39.6%)   
                                              2 - bac        1.0 (20.0%)   
                                              3 - sup bac    4.6 (64.8%)   
                                              All           11.3 (42.6%)   

vaccincovid                                                          Oui  \
PROXPARTI                                     diplome                      
1 - Centre (p = 0.204)                        1 - inf bac   94.7 (82.4%)   
                                              2 - bac       28.2 (69.8%)   
                                              3 - sup bac  103.5 (81.2%)   
                                              All          226.4 (80.1%)   
2 - Aucune formation politique (p = 0.657)    1 - inf bac  219.7 (55.0%)   
                                              2 - bac       82.4 (56.7%)   
                                              3 - sup bac  116.5 (59.0%)   
                                              All          418.7 (56.4%)   
3 - Gauche (p = 0.800)                        1 - inf bac   72.1 (71.4%)   
                                              2 - bac       26.6 (66.3%)   
                                              3 - sup bac   67.4 (68.0%)   
                                              All          166.1 (69.2%)   
4 - Droite (p = 0.003)                        1 - inf bac   87.1 (89.2%)   
                                              2 - bac       30.9 (65.7%)   
                                              3 - sup bac   65.0 (76.1%)   
                                              All          182.9 (79.5%)   
5 - Extrême gauche (p = 0.755)                1 - inf bac   64.4 (64.1%)   
                                              2 - bac       29.7 (60.4%)   
                                              3 - sup bac   28.0 (58.0%)   
                                              All          122.1 (61.7%)   
6 - Extrême droite (p = 0.446)                1

In [24]:
pyshs.tableau_croise_controle(data1,"PROXPARTI", "diplome","vaccin","RED").sort_index()

vaccin                                                               Non  \
PROXPARTI                                     diplome                      
1 - Centre (p = 0.205)                        1 - inf bac   17.4 (15.2%)   
                                              2 - bac        8.3 (20.5%)   
                                              3 - sup bac   12.9 (10.1%)   
                                              All           38.6 (13.7%)   
2 - Aucune formation politique (p = 0.004)    1 - inf bac  151.1 (37.9%)   
                                              2 - bac       43.6 (30.0%)   
                                              3 - sup bac   48.7 (24.7%)   
                                              All          243.4 (32.8%)   
3 - Gauche (p = 0.248)                        1 - inf bac   24.5 (24.3%)   
                                              2 - bac        9.8 (24.4%)   
                                              3 - sup bac   15.3 (15.4%)   
                                              All           49.6 (20.6%)   
4 - Droite (p = 0.680)                        1 - inf bac     8.9 (9.1%)   
                                              2 - bac        5.8 (12.3%)   
                                              3 - sup bac     6.6 (7.7%)   
                                              All            21.3 (9.3%)   
5 - Extrême gauche (p = 0.893)                1 - inf bac   30.8 (30.6%)   
                                              2 - bac       13.3 (27.0%)   
                                              3 - sup bac   14.7 (30.5%)   
                                              All           58.8 (29.7%)   
6 - Extrême droite (p = 0.880)                1 - inf bac   70.5 (37.1%)   
                                              2 - bac       19.7 (41.0%)   
                                              3 - sup bac   18.9 (38.7%)   
                                              All          109.1 (38.1%)   
7 - Une autre formation politique (p = 0.237) 1 - inf bac     1.1 (7.7%)   
                                              2 - bac        0.5 (10.0%)   
                                              3 - sup bac    2.5 (35.2%)   
                                              All            4.1 (15.5%)   

vaccin                                                               Oui  \
PROXPARTI                                     diplome                      
1 - Centre (p = 0.205)                        1 - inf bac   97.4 (84.8%)   
                                              2 - bac       32.2 (79.5%)   
                                              3 - sup bac  114.5 (89.9%)   
                                              All          244.1 (86.3%)   
2 - Aucune formation politique (p = 0.004)    1 - inf bac  248.1 (62.1%)   
                                              2 - bac      101.7 (70.0%)   
                                              3 - sup bac  148.8 (75.3%)   
                                              All          498.6 (67.2%)   
3 - Gauche (p = 0.248)                        1 - inf bac   76.5 (75.7%)   
                                              2 - bac       30.4 (75.6%)   
                                              3 - sup bac   83.8 (84.6%)   
                                              All          190.6 (79.4%)   
4 - Droite (p = 0.680)                        1 - inf bac   88.8 (90.9%)   
                                              2 - bac       41.2 (87.7%)   
                                              3 - sup bac   78.8 (92.3%)   
                                              All          208.7 (90.7%)   
5 - Extrême gauche (p = 0.893)                1 - inf bac   69.7 (69.4%)   
                                              2 - bac       35.9 (73.0%)   
                                              3 - sup bac   33.5 (69.5%)   
                                              All          139.1 (70.3%)   
6 - Extrême droite (p = 0.880)                1

Confiance dans le ministère pour la sécu des vaccins

In [28]:
pyshs.tableau_croise_controle(data1,"PROXPARTI", "confMinistereSecu","vaccincovid","RED").sort_index()

vaccincovid                                                                Non  \
PROXPARTI                                     confMinistereSecu                  
1 - Centre (p = 0.000)                        1 - D'accord        24.3 (11.1%)   
                                              2 - Pas d'accord    26.0 (50.4%)   
                                              3 - NSP              6.0 (48.4%)   
                                              All                 56.4 (19.9%)   
2 - Aucune formation politique (p = 0.000)    1 - D'accord        60.7 (20.9%)   
                                              2 - Pas d'accord   196.6 (62.2%)   
                                              3 - NSP             66.0 (48.9%)   
                                              All                323.3 (43.6%)   
3 - Gauche (p = 0.000)                        1 - D'accord        24.6 (19.0%)   
                                              2 - Pas d'accord    45.4 (47.2%)   
                                              3 - NSP              4.1 (28.3%)   
                                              All                 74.1 (30.8%)   
4 - Droite (p = 0.052)                        1 - D'accord        23.4 (16.8%)   
                                              2 - Pas d'accord    22.6 (29.0%)   
                                              3 - NSP               1.0 (7.7%)   
                                              All                 47.1 (20.5%)   
5 - Extrême gauche (p = 0.000)                1 - D'accord         9.7 (14.7%)   
                                              2 - Pas d'accord    62.1 (56.1%)   
                                              3 - NSP              4.0 (18.8%)   
                                              All                 75.8 (38.3%)   
6 - Extrême droite (p = 0.000)                1 - D'accord        28.8 (34.1%)   
                                              2 - Pas d'accord   100.0 (60.4%)   
                                              3 - NSP             25.4 (69.2%)   
                                              All                154.2 (53.8%)   
7 - Une autre formation politique (p = 0.013) 1 - D'accord         1.8 (14.1%)   
                                              2 - Pas d'accord     9.2 (71.3%)   
                                              3 - NSP              0.3 (37.5%)   
                                              All                 11.3 (42.6%)   

vaccincovid                                                                Oui  \
PROXPARTI                                     confMinistereSecu                  
1 - Centre (p = 0.000)                        1 - D'accord       194.5 (88.9%)   
                                              2 - Pas d'accord    25.6 (49.6%)   
                                              3 - NSP              6.4 (51.6%)   
                                              All                226.4 (80.1%)   
2 - Aucune formation politique (p = 0.000)    1 - D'accord       230.3 (79.1%)   
                                              2 - Pas d'accord   119.3 (37.8%)   
                                              3 - NSP             69.1 (51.1%)   
                                              All                418.7 (56.4%)   
3 - Gauche (p = 0.000)                        1 - D'accord       105.0 (81.0%)   
                                              2 - Pas d'accord    50.7 (52.8%)   
                                              3 - NSP             10.4 (71.7%)   
                                              All                166.1 (69.2%)   
4 - Droite (p = 0.052)                        1 - D'accord       115.5 (83.2%)   
                                              2 - Pas d'accord    55.4 (71.0%)   
                                              3 - NSP             12.0 (92.3%)   
                                              All                182.9 (79.5%)   
5 - Extrême gauche (p = 0.000)                1 - D'accord     

In [29]:
pyshs.tableau_croise_controle(data1,"PROXPARTI", "confMinistereSecu","vaccin","RED").sort_index()

vaccin                                                                     Non  \
PROXPARTI                                     confMinistereSecu                  
1 - Centre (p = 0.000)                        1 - D'accord         15.9 (7.3%)   
                                              2 - Pas d'accord    20.9 (40.5%)   
                                              3 - NSP              1.8 (14.5%)   
                                              All                 38.6 (13.7%)   
2 - Aucune formation politique (p = 0.000)    1 - D'accord        46.0 (15.8%)   
                                              2 - Pas d'accord   140.0 (44.3%)   
                                              3 - NSP             57.5 (42.6%)   
                                              All                243.4 (32.8%)   
3 - Gauche (p = 0.000)                        1 - D'accord         11.7 (9.0%)   
                                              2 - Pas d'accord    30.9 (32.2%)   
                                              3 - NSP              6.9 (47.9%)   
                                              All                 49.6 (20.6%)   
4 - Droite (p = 0.000)                        1 - D'accord          4.1 (3.0%)   
                                              2 - Pas d'accord    16.6 (21.3%)   
                                              3 - NSP               0.5 (3.8%)   
                                              All                  21.3 (9.3%)   
5 - Extrême gauche (p = 0.000)                1 - D'accord         6.8 (10.3%)   
                                              2 - Pas d'accord    47.4 (42.8%)   
                                              3 - NSP              4.6 (21.6%)   
                                              All                 58.8 (29.7%)   
6 - Extrême droite (p = 0.001)                1 - D'accord        18.3 (21.7%)   
                                              2 - Pas d'accord    71.3 (43.1%)   
                                              3 - NSP             19.4 (52.9%)   
                                              All                109.1 (38.1%)   
7 - Une autre formation politique (p = 0.318) 1 - D'accord          0.7 (5.5%)   
                                              2 - Pas d'accord     3.4 (26.4%)   
                                              3 - NSP               0.0 (0.0%)   
                                              All                  4.1 (15.5%)   

vaccin                                                                     Oui  \
PROXPARTI                                     confMinistereSecu                  
1 - Centre (p = 0.000)                        1 - D'accord       202.8 (92.7%)   
                                              2 - Pas d'accord    30.7 (59.5%)   
                                              3 - NSP             10.6 (85.5%)   
                                              All                244.1 (86.3%)   
2 - Aucune formation politique (p = 0.000)    1 - D'accord       245.0 (84.2%)   
                                              2 - Pas d'accord   175.9 (55.7%)   
                                              3 - NSP             77.6 (57.4%)   
                                              All                498.6 (67.2%)   
3 - Gauche (p = 0.000)                        1 - D'accord       117.9 (91.0%)   
                                              2 - Pas d'accord    65.2 (67.8%)   
                                              3 - NSP              7.5 (52.1%)   
                                              All                190.6 (79.4%)   
4 - Droite (p = 0.000)                        1 - D'accord       134.8 (97.0%)   
                                              2 - Pas d'accord    61.4 (78.7%)   
                                              3 - NSP             12.5 (96.2%)   
                                              All                208.7 (90.7%)   
5 - Extrême gauche (p = 0.000)                1 - D'accord     

Confiance dans le ministère pour l'info sur les vaccins

In [30]:
pyshs.tableau_croise_controle(data1,"PROXPARTI", "confMinistereInfo","vaccincovid","RED").sort_index()

vaccincovid                                                                Non  \
PROXPARTI                                     confMinistereInfo                  
1 - Centre (p = 0.000)                        1 - Confiance       25.8 (11.4%)   
                                              2 - Pas confiance   30.5 (54.9%)   
                                              All                 56.4 (19.9%)   
2 - Aucune formation politique (p = 0.000)    1 - Confiance       92.3 (24.4%)   
                                              2 - Pas confiance  231.0 (63.5%)   
                                              All                323.3 (43.6%)   
3 - Gauche (p = 0.000)                        1 - Confiance       27.7 (19.3%)   
                                              2 - Pas confiance   46.4 (48.0%)   
                                              All                 74.1 (30.8%)   
4 - Droite (p = 0.012)                        1 - Confiance       25.1 (15.7%)   
                                              2 - Pas confiance   22.0 (31.2%)   
                                              All                 47.1 (20.5%)   
5 - Extrême gauche (p = 0.000)                1 - Confiance       19.9 (23.6%)   
                                              2 - Pas confiance   55.9 (49.2%)   
                                              All                 75.8 (38.3%)   
6 - Extrême droite (p = 0.000)                1 - Confiance       36.7 (38.1%)   
                                              2 - Pas confiance  117.5 (61.7%)   
                                              All                154.2 (53.8%)   
7 - Une autre formation politique (p = 0.094) 1 - Confiance        1.1 (13.1%)   
                                              2 - Pas confiance   10.2 (56.4%)   
                                              All                 11.3 (42.6%)   

vaccincovid                                                                Oui  \
PROXPARTI                                     confMinistereInfo                  
1 - Centre (p = 0.000)                        1 - Confiance      201.3 (88.6%)   
                                              2 - Pas confiance   25.1 (45.1%)   
                                              All                226.4 (80.1%)   
2 - Aucune formation politique (p = 0.000)    1 - Confiance      286.2 (75.6%)   
                                              2 - Pas confiance  132.5 (36.5%)   
                                              All                418.7 (56.4%)   
3 - Gauche (p = 0.000)                        1 - Confiance      115.8 (80.7%)   
                                              2 - Pas confiance   50.3 (52.0%)   
                                              All                166.1 (69.2%)   
4 - Droite (p = 0.012)                        1 - Confiance      134.5 (84.3%)   
                                              2 - Pas confiance   48.4 (68.8%)   
                                              All                182.9 (79.5%)   
5 - Extrême gauche (p = 0.000)                1 - Confiance       64.3 (76.4%)   
                                              2 - Pas confiance   57.8 (50.8%)   
                                              All                122.1 (61.7%)   
6 - Extrême droite (p = 0.000)                1 - Confiance       59.5 (61.9%)   
                                              2 - Pas confiance   72.9 (38.3%)   
                                              All                132.5 (46.2%)   
7 - Une autre formation politique (p = 0.094) 1 - Confiance        7.3 (86.9%)   
                                              2 - Pas confiance    7.9 (43.6%)   
                                              All                 15.2 (57.4%)   

vaccincovid                                                               All  
PROXPARTI                                     confMinistereInfo                
1 - Centre (p = 0.000)                        1 - Confiance      2

In [31]:
pyshs.tableau_croise_controle(data1,"PROXPARTI", "confMinistereInfo","vaccin","RED").sort_index()

vaccin                                                                     Non  \
PROXPARTI                                     confMinistereInfo                  
1 - Centre (p = 0.000)                        1 - Confiance        14.5 (6.4%)   
                                              2 - Pas confiance   24.1 (43.3%)   
                                              All                 38.6 (13.7%)   
2 - Aucune formation politique (p = 0.000)    1 - Confiance       77.1 (20.4%)   
                                              2 - Pas confiance  166.3 (45.7%)   
                                              All                243.4 (32.8%)   
3 - Gauche (p = 0.000)                        1 - Confiance        11.6 (8.1%)   
                                              2 - Pas confiance   38.0 (39.3%)   
                                              All                 49.6 (20.6%)   
4 - Droite (p = 0.000)                        1 - Confiance         6.6 (4.1%)   
                                              2 - Pas confiance   14.6 (20.7%)   
                                              All                  21.3 (9.3%)   
5 - Extrême gauche (p = 0.001)                1 - Confiance       14.3 (17.0%)   
                                              2 - Pas confiance   44.5 (39.1%)   
                                              All                 58.8 (29.7%)   
6 - Extrême droite (p = 0.001)                1 - Confiance       22.9 (23.8%)   
                                              2 - Pas confiance   86.1 (45.2%)   
                                              All                109.1 (38.1%)   
7 - Une autre formation politique (p = 0.356) 1 - Confiance         0.0 (0.0%)   
                                              2 - Pas confiance    4.1 (22.7%)   
                                              All                  4.1 (15.5%)   

vaccin                                                                     Oui  \
PROXPARTI                                     confMinistereInfo                  
1 - Centre (p = 0.000)                        1 - Confiance      212.7 (93.6%)   
                                              2 - Pas confiance   31.5 (56.7%)   
                                              All                244.1 (86.3%)   
2 - Aucune formation politique (p = 0.000)    1 - Confiance      301.4 (79.6%)   
                                              2 - Pas confiance  197.2 (54.3%)   
                                              All                498.6 (67.2%)   
3 - Gauche (p = 0.000)                        1 - Confiance      131.9 (91.9%)   
                                              2 - Pas confiance   58.7 (60.7%)   
                                              All                190.6 (79.4%)   
4 - Droite (p = 0.000)                        1 - Confiance      152.9 (95.9%)   
                                              2 - Pas confiance   55.8 (79.3%)   
                                              All                208.7 (90.7%)   
5 - Extrême gauche (p = 0.001)                1 - Confiance       70.0 (83.0%)   
                                              2 - Pas confiance   69.2 (60.9%)   
                                              All                139.1 (70.3%)   
6 - Extrême droite (p = 0.001)                1 - Confiance       73.3 (76.2%)   
                                              2 - Pas confiance  104.3 (54.8%)   
                                              All                177.6 (61.9%)   
7 - Une autre formation politique (p = 0.356) 1 - Confiance       8.4 (100.0%)   
                                              2 - Pas confiance   14.0 (77.3%)   
                                              All                 22.4 (84.5%)   

vaccin                                                                    All  
PROXPARTI                                     confMinistereInfo                
1 - Centre (p = 0.000)                        1 - Confiance      2